In [1]:
#import fwiVis.fwiVis as fv
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
from math import cos, asin, sqrt
import re

import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import rasterio
import glob
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings
import folium
import datetime
import time
from folium import plugins
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
import contextily as cx
from shapely.geometry import box
import sys
from datetime import datetime, timedelta
from itertools import chain

from datetime import date


sys.path.insert(0, '/projects/old_shared/fire_weather_vis/base-fwi-vis/')
import fwiVis.fwiVis as fv

In [ ]:
abfa = gpd.read_file("/projects/old_shared/fire_weather_vis/Lightning_analysis/ABFA/final_viirs2023.gpkg")

In [ ]:
abfa = abfa.cx[-48.45463578921794:-83.69877641421793, 62.94135765648493:44.25483911637959]

In [ ]:
abfa.explore()

In [7]:
def prep_fire_files(path, crs = "3571"):
    '''
    Read in cvs of previously output GeoDataFrame as a GeoDataFrame with valid geometry column. 
    '''
    fires = pd.read_csv(path)
    fires = fires.rename(columns={"geometry":"csv_geometry"})
    fires.t = fires.t.astype("str")
    fires.fireID  = fires.fireID.astype("str")
    #fires['csv_geometry'] =fires['csv_geometry'].apply(wkt.loads)
    fires_geom = gpd.read_file(path, GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")
    fires_geom.t = fires_geom.t.astype("str")
    fires_geom.fireID  = fires_geom.fireID.astype("str")
    fires = fires_geom[["fireID", "t", "geometry"]].merge(fires, on=["fireID", "t"], how = "left")
    fires = fires.set_crs(crs)
    return (fires)


path = "/projects/old_shared/fire_weather_vis/Lightning_analysis/fwi_timeline_merge/Final_dataset_as_of_20240209.csv" 
fires = prep_fire_files(path)

In [10]:
fires.t = fires.t.astype("str")

In [ ]:
fires[["fireID", "geometry", "t"]].explore(style_kwds = {"fillOpacity": 0})